In [39]:
# all imports
import numpy as np
import os
import requests
from astropy.table import Table, vstack
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.wcs import WCS
import math
import matplotlib as mpl
from astropy.visualization.wcsaxes import SphericalCircle
import pandas as pd

In [8]:
# define directories
data_dir = '/global/cfs/projectdirs/desi/science/td/pv/tfgalaxies/'
cache_dir = '/pscratch/sd/j/jjpim/cache/' # my cache path
vi_dir = '/pscratch/sd/j/jjpim/visual_inspection/' # vi image path

In [17]:
# load updated fuji table
tf_fuji = Table.read(data_dir + 'SV/desi_pv_tf_fuji_healpix_rotcurve_EOA_VI.fits') 
tf_fuji[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
1079550234591232,194.390863195343,27.5157211790145,10378,sv3,1.1235686466514,7.31685779475115e-05,4,3.28414569795132,fuji/healpix/sv3/bright/103/10378/redrock-sv3-bright-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124544,194.390863195343,27.5157211790145,10378,sv3,0.686773088332363,6.9756676262104e-05,4,0.786607094109058,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.39086319534337,27.51572117901454,1.0,0.03927420444753712,1.0000000170652443,0.0,0.0
1092744374124546,194.364461113654,27.5037185881314,10378,sv3,0.0242933923052181,4.95233472646785e-05,0,95.428411073226,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.36446111365385,27.50371858813136,1.0,0.012960487467698754,0.3300000056318236,0.0,0.0
1092744369930240,194.338458724402,27.4918902690326,10378,sv3,0.0264170223697961,0.00010139452689994,0,9.53278421035066,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.33845872440244,27.491890269032595,1.0,0.012960487467685656,0.33000000563149007,0.0,0.0
1092744374124545,194.377858465028,27.5098100780282,10378,sv3,0.211332646769145,6.68535116703737e-05,4,3.73989077657461,fuji/healpix/sv3/dark/103/10378/redrock-sv3-dark-10378.fits,EXT,662902,194.3778584650283,27.509810078028195,1.0,0.026313716979861984,0.6700000114340222,0.0,0.0


In [18]:
# load iron table
tf_iron = Table.read(data_dir + 'Y1/desi_pv_tf_iron_healpix_rotcurve_EOA_VI.fits')
tf_iron[:5]

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64
2852147603439621,198.369130660983,36.5372037049171,10475,main,0.815976335547845,7.38513168100107e-05,4,0.128754377365112,iron/healpix/main/dark/104/10475/redrock-main-dark-10475.fits,EXT,649377,198.36913066098333,36.537203704917076,0.0,0.0567420462925582,0.599999979158402,0.0,1.0
2399148812795907,198.371733180003,36.4994335406917,10475,main,1.11088784970434,7.48767797671894e-05,4,7.9473560154438,iron/healpix/main/bright/104/10475/redrock-main-bright-10475.fits,EXT,649377,198.37173318000336,36.499433540691676,0.0,0.09457007715465947,0.9999999652681886,0.0,1.0
2399382443917318,184.845242475328,49.8157304793777,10995,main,1.14739342108157,0.000146302276719084,4,2.56771463155746,iron/healpix/main/bright/109/10995/redrock-main-bright-10995.fits,EXT,1008911,184.84524247532795,49.81573047937771,0.0,0.024841917763806447,0.9999999684347345,0.0,0.0
2399634072797192,184.341289722203,70.8283725474297,11965,main,1.51703376230705,6.28979649962091e-05,4,4.76254060305655,iron/healpix/main/bright/119/11965/redrock-main-bright-11965.fits,EXT,241234,184.34128972220284,70.82837254742968,0.0,0.026555350833845817,0.8000000542794348,0.0,0.0
2852141710442505,123.256011148025,36.2652948002806,6448,main,0.00787379494184006,3.4714052819995e-05,0,22.1719104201402,iron/healpix/main/dark/64/6448/redrock-main-dark-6448.fits,EXT,31591,123.25601114802525,36.26529480028061,0.0,0.015878717350385107,0.40000000619171416,0.0,0.0


In [23]:
tf_fuji['VELOCITY']=np.nan
tf_fuji['VELOCITY_REL']=np.nan
tf_iron['VELOCITY']=np.nan
tf_iron['VELOCITY_REL']=np.nan

In [47]:
# Filter the data based on the condition
filtered_tf_fuji = tf_fuji[tf_fuji['SKY_FIBER_DIST_R26'] < 0.001]
filtered_tf_iron = tf_iron[tf_iron['SKY_FIBER_DIST_R26'] < 0.001]

filtered_tf_fuji = filtered_tf_fuji[filtered_tf_fuji['ROT_CURVE'] == 1]
filtered_tf_iron = filtered_tf_iron[filtered_tf_iron['ROT_CURVE'] == 1]

# Find the maximum value of a specific column in the filtered data
max_value_fuji = filtered_tf_fuji['Z'].max()
max_value_iron = filtered_tf_iron['Z'].max()

print("Maximum value in tf_fuji:", max_value_fuji)
print("Maximum value in tf_iron:", max_value_iron)

Maximum value in tf_fuji: 0.173630148670864
Maximum value in tf_iron: 1.62543750380985


In [63]:
tf_fuji_rot = np.unique(tf_fuji[tf_fuji['ROT_CURVE']==1]['SGA_ID'])
tf_iron_rot = np.unique(tf_iron[tf_iron['ROT_CURVE']==1]['SGA_ID'])
print(tf_fuji_rot, tf_iron_rot)

 SGA_ID
-------
    123
    896
   1548
   1583
   1980
   2497
   5732
   5958
   5964
   6443
    ...
1421164
1421334
1424489
1427956
1429142
1429680
1430013
1431285
1434487
5001339
5002047
Length = 1267 rows  SGA_ID
-------
    123
    896
   1548
   1583
   1980
   2497
   5732
   5958
   5964
   6443
    ...
1421334
1424489
1427956
1429142
1429680
1430013
1431285
1434487
2000055
5001339
5002047
Length = 1298 rows


In [155]:
# find all unique target ID's for one galaxy
targets_fuji=tf_fuji[tf_fuji['SGA_ID']==  123]
targets_iron=tf_iron[tf_iron['SGA_ID']== 123]
targets=(vstack([targets_fuji,targets_iron]))
target_list=np.unique(targets['TARGETID'])
print(target_list)

     TARGETID    
-----------------
 1071233026228227
 1071233026228228
 1071233026228231
 1084427165761544
39633304835591852


In [156]:
z_targets=(targets)
print(z_targets)

     TARGETID        TARGET_RA        TARGET_DEC    ... VELOCITY VELOCITY_REL
----------------- ---------------- ---------------- ... -------- ------------
 1071233026228227 242.564878755415 53.6763270782975 ...      nan          nan
 1071233026228228 242.564908359756 53.6746327647682 ...      nan          nan
 1071233026228231 242.565028546859 53.6677528249076 ...      nan          nan
 1084427165761544 242.565058139153 53.6660585113414 ...      nan          nan
39633304835591852 242.564968458214  53.671192794853 ...      nan          nan
39633304835591852 242.564968458214  53.671192794853 ...      nan          nan


In [113]:
#list of good centers in fuji for one galaxy found by SGA ID above
fuji_centers = targets_fuji[targets_fuji['SKY_FIBER_DIST_R26'] < 0.001]
fuji_centers = fuji_centers[fuji_centers['ZWARN']==0]
fuji_good_centers = fuji_centers[fuji_centers['DELTACHI2']>25] 
fuji_good_centers

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY,VELOCITY_REL
int64,float64,float64,int64,bytes3,float64,float64,int64,float64,bytes63,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109965464429358,1.30236298072821e-05,0,847.705579817295,fuji/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,nan,nan


In [112]:
#list of good centers in iron for one galaxy found by SGA ID above
iron_centers = targets_iron[targets_iron['SKY_FIBER_DIST_R26'] < 0.001]
iron_centers = iron_centers[iron_centers['ZWARN']==0]
iron_good_centers = iron_centers[iron_centers['DELTACHI2']>25] 
iron_good_centers

TARGETID,TARGET_RA,TARGET_DEC,HEALPIX,SURVEY,Z,ZERR,ZWARN,DELTACHI2,FILENAME,PVTYPE,SGA_ID,RA,DEC,OBS_IN_SV,SKY_FIBER_DIST,SKY_FIBER_DIST_R26,ROT_CURVE,EOA,VELOCITY_REL,VELOCITY
int64,float64,float64,int64,bytes4,float64,float64,int64,float64,bytes65,bytes3,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64
39633304835591852,242.564968458214,53.671192794853,10153,sv3,0.109966732247689,1.26106529808024e-05,0,879.694150358438,iron/healpix/sv3/bright/101/10153/redrock-sv3-bright-10153.fits,TFT,123,242.56496845821405,53.67119279485298,1.0,1.6837541040866038e-14,3.2792580096330368e-12,1.0,0.0,nan,nan


In [157]:
z_o=iron_good_centers['Z']
z_o

0.109966732247689
